In [2]:
from data_loader.MTL_dataloader import get_train_validation_generator
from MTL_model import MTL_model as MTL_model_class
from callback import CustomCallBacks
from evaluation.evaluate import evaluate
from tensorflow.keras.optimizers import Adam

# Data loader

In [ ]:
det_csv_path = "/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv"
seg_csv_path = "/kaggle/input/siim-acr-pneumothorax-segmentation-data/train-rle.csv"
det_images_path = "/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/"
seg_images_path = "/kaggle/input/siim-acr-pneumothorax-segmentation-data/dicom-images-train/"

In [ ]:
train_gen,val_gen = get_train_validation_generator(det_csv_path,seg_csv_path , det_images_path, seg_images_path)

# Build Model

In [ ]:
MTL_model_clss= MTL_model_class()
MTL_model = MTL_model_clss.MTL_model

In [ ]:
#from tensorflow.keras.utils import plot_model
#plot_model(MTL_model.MTL_model)

In [ ]:
#MTL_model.MTL_model.summary()

# Train Model

In [ ]:
INIT_LR = 1e-4
EPOCHS =20
model_weights_path = "model_weights.hdf5"

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

In [ ]:
lossWeights = [1.0,1.0,1.0]
MTL_model_clss.MTL_model.compile(optimizer=opt, loss=mtl_loss,loss_weights=lossWeights,  metrics=[])

In [ ]:
history = MTL_model_clss.MTL_model.fit_generator(train_gen,validation_data=val_gen,  callbacks=CustomCallBacks() )

In [ ]:
MTL_model_clss.MTL_model.save_weights(model_weights_path)

# Evaluate

In [ ]:
MTL_model_clss.MTL_model.load_weights(model_weights_path)

In [ ]:
mAP,dice_coef,f1,per,rec,acc = evaluate(val_gen,MTL_model_clss.MTL_model,MTL_model_clss.MTL_model, MTL_model_clss.detector.anchors )